In [130]:
from os import listdir
import pandas as pd
import numpy as np

from nltk.metrics.distance import edit_distance

In [106]:
MISSING = " "

# Load Data

In [107]:
files = listdir("../chart")

dfs_chart = {}
for f in files:
    year = f.split("_")[0]
    month = f.split("_")[1].split(".")[0]

    if dfs_chart.get(year) is None:
        dfs_chart[year] = {}
        
    dfs_chart[year][month] = pd.read_csv("../chart/" + f)

In [108]:
dfs_chart["2006"]["Janvier"]

,Rank,Artist,Music,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays
0,1,Juanes,La Camisa Negra,Juanes,"9 août 1972 (50 ans)Medellin, Colombie",NaN,NaN,NaN,NaN
1,2,Madonna,Hung Up,Madonna,"16 août 1958 (64 ans)Bay City, Michigan (États...",NaN,NaN,Américaine,NaN
2,3,Johnny Hallyday,Mon Plus Beau Noël,Johnny Hallyday,15 juin 1943 Paris 9e (France),NaN,NaN,Française,NaN
3,4,Tina Arena,Aimer Jusqu'à L'impossible,Tina Arena,"1er novembre 1967 (55 ans)Melbourne, Australie",NaN,NaN,NaN,NaN
4,5,Star Academy,Santiano,Star Academy (France),NaN,NaN,NaN,NaN,France
...,...,...,...,...,...,...,...,...,...
95,96,Jenifer,Serre-moi,Jenifer,15 novembre 1982 (40 ans)Nice,NaN,NaN,NaN,NaN
96,97,Lucie Bernardoni,"Petit Rat, Petit Loup",Lucie Bernardoni,"4 février 1987 (35 ans)Nice, Alpes-Maritimes, ...",NaN,NaN,NaN,NaN
97,98,Ilona Mitrecey,C'est Les Vacances,Ilona Mitrecey,1er septembre 1993 (29 ans)Fontenay-aux-Roses ...,NaN,NaN,NaN,NaN
98,99,Kanye West,Gold Digger,Kanye West,"8 juin 1977 (45 ans)Atlanta, Géorgie (États-Unis)",NaN,NaN,NaN,NaN


In [109]:
artist = pd.read_csv("../artist.csv", index_col=0)

In [110]:
artist

,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays
Artist,,,,,,
Lady GaGa,Lady Gaga,"28 mars 1986 (36 ans)New York, État de New Yor...",NaN,NaN,NaN,NaN
Starley,Starley,3 octobre 1987 (35 ans)Sydney,NaN,NaN,australienne,NaN
Dj Sem,Lacrim,"19 avril 1985 (37 ans)Paris (20e), France",NaN,NaN,NaN,NaN
Ckay,CKay,16 juillet 1995 (27 ans)Kaduna,NaN,NaN,nigériane,NaN
Ray Charles,Ray Charles,"23 septembre 1930Albany ( Géorgie, États-Unis)",NaN,NaN,américaine,NaN
...,...,...,...,...,...,...
Hamza,Hamza (rappeur),"1er août 1994 (28 ans)Laeken, Belgique",NaN,NaN,NaN,NaN
Alvaro Soler,Álvaro Soler,"9 janvier 1991 (31 ans)Barcelone, Espagne",NaN,NaN,NaN,NaN
Michael Bublé,Michael Bublé,9 septembre 1975 (47 ans)Burnaby (Colombie-Bri...,NaN,NaN,NaN,NaN


# Analysis

In [116]:
artist["distance"] = artist.apply(lambda x: edit_distance(x.name, str(x["Artist_wiki"])), axis=1)

In [150]:
artist[artist.isna().sum(axis=1) >= 5]

,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays,distance
Artist,,,,,,,
Hi-tack,Occitan,NaN,NaN,NaN,NaN,NaN,6
College,Collège,NaN,NaN,NaN,NaN,NaN,1
Ambiance Skandal,Offishal,NaN,NaN,NaN,NaN,NaN,13
Gnash,Gnash,NaN,NaN,NaN,NaN,NaN,0
St Ange,Archange,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...
Pat,Pat,NaN,NaN,NaN,NaN,NaN,0
Boris,Boris,NaN,NaN,NaN,NaN,NaN,0
Fleur De Vanille,Fleur blanche,NaN,NaN,NaN,NaN,NaN,7


In [127]:
artist["Missing"] = arti

,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays,distance
Artist,,,,,,,
Lady GaGa,Lady Gaga,"28 mars 1986 (36 ans)New York, État de New Yor...",NaN,NaN,NaN,NaN,1
Starley,Starley,3 octobre 1987 (35 ans)Sydney,NaN,NaN,australienne,NaN,0
Dj Sem,Lacrim,"19 avril 1985 (37 ans)Paris (20e), France",NaN,NaN,NaN,NaN,5
Ckay,CKay,16 juillet 1995 (27 ans)Kaduna,NaN,NaN,nigériane,NaN,1
Ray Charles,Ray Charles,"23 septembre 1930Albany ( Géorgie, États-Unis)",NaN,NaN,américaine,NaN,0
...,...,...,...,...,...,...,...
Hamza,Hamza (rappeur),"1er août 1994 (28 ans)Laeken, Belgique",NaN,NaN,NaN,NaN,10
Alvaro Soler,Álvaro Soler,"9 janvier 1991 (31 ans)Barcelone, Espagne",NaN,NaN,NaN,NaN,1
Michael Bublé,Michael Bublé,9 septembre 1975 (47 ans)Burnaby (Colombie-Bri...,NaN,NaN,NaN,NaN,0
